In [1]:
import numpy as np
import sys
sys.path.append("..")
import pprint
from reference.lib.envs.gridworld import GridworldEnv

In [4]:
env = GridworldEnv()

In [56]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.0000001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    # loop until convergence - system is stable
    while True:
        # this is the change value
        delta_value = 0
        
        # for every state value
        for i in range(len(V)):
            
            # temp variable reset to zero
            current_state = V[i]
            V[i] = 0
            # for every action value - here 4
            for policy_index in range(4):
                # choose the action and prob value
                action = policy_index
                action_prob = policy[i,policy_index]
                # this is not the optimal value
                for prob, next_state, reward, done in env.P[i][action]:
                    V[i] = V[i] + action_prob * prob * (reward + discount_factor * V[next_state]) 
            # stopping critiria
            delta_value = max(delta_value,np.abs(current_state-V[i]))
            #V[i] = current_state
            
        if delta_value < theta: 
            break
            
    return np.array(V)

In [57]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
print(v.reshape((4,4)))

[[ 0.         -4.01259081 -4.65070572 -3.48647892]
 [-5.99715026 -7.39965764 -7.10375324 -5.04520998]
 [-7.99122337 -8.48513641 -7.3194397  -4.46898804]
 [-7.28962827 -7.23022505 -5.21988117  0.        ]]


In [54]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
print(expected_v.reshape((4,4)))

[[  0 -14 -20 -22]
 [-14 -18 -20 -20]
 [-20 -20 -18 -14]
 [-22 -20 -14   0]]


In [55]:
print(expected_v/(v+1e-8))

[0.         1.00000005 1.00000005 1.00000005 1.00000005 1.00000005
 1.00000005 1.00000005 1.00000005 1.00000005 1.00000004 1.00000004
 1.00000005 1.00000005 1.00000004 0.        ]


In [34]:
# test 
print(
    np.testing.assert_array_almost_equal(v, expected_v, decimal=2)
)

None
